In [21]:
from fonctions.gestion_bdd import lire_bdd_perso
import pandas as pd
import numpy as np

In [22]:
saison = 13
joueur = 'tomlora'
nb_champ_max = 5

In [23]:
df = lire_bdd_perso(f'''SELECT matchs.*, tracker.discord from matchs
                    INNER JOIN tracker on matchs.joueur = tracker.index 
                    WHERE tracker.activation = true and season = {saison}''',
                    index_col = 'match_id').T

In [24]:
df_ranked = df[df['mode'] == 'RANKED']

df_ranked = df_ranked[df_ranked['joueur'] == joueur]

def creation_agregat(df):
    
    for compte in df['joueur'].unique():
        print(compte)
        
        
        discord_id = df.loc[df['joueur'] == compte, 'discord'].unique()[0]
        df_filter = df[df['discord'] == discord_id]
        
        print(discord_id)
        
        def feature_engineering(df):
            df['afk'] = df['afk'].astype(int)
            return df
        
        df_filter = feature_engineering(df_filter)
        
        liste_champ = {'kills' : ['sum', 'mean'],
                        'assists' : ['sum', 'mean'],
                        'deaths' : ['sum', 'mean'],
                        'double' : ['sum'],
                        'triple' : ['sum'],
                        'quadra' : ['sum'],
                        'penta' : ['sum'],
                        'time' : ['sum', 'mean'],
                        'victoire' : ['sum', 'count'],
                        'cs_min' : ['sum', 'mean'],
                        'vision_avantage' : ['mean'],
                        'vision_min' : ['mean'],
                        'solokills' : ['sum'],
                        'serie_kills' : ['max'],
                        'drake' : ['sum'],
                        'baron' : ['sum'],
                        'afk' : ['sum'],
                        'temps_dead' : ['sum', 'mean'], # Sur time, tu as passé X minutes morts
                        'kp' : ['mean'],
                        'kda' : ['sum', 'mean'],
                        'lp' : ['sum'],
                        'temps_avant_premiere_mort' : ['mean'],
                        'couronne' : ['sum', 'mean'],
                        'team' : ['sum'], # nombre de redside. A comparer avec le nombre de game
                        'team_kills' : ['sum', 'mean'],
                        'team_deaths' : ['sum', 'mean'],
                        'champion' : ['count'] # nombre de games
                        }


        df_grp = df_filter.groupby(['joueur', 'role']).agg(liste_champ)
                
        df_grp_champ = df_filter.groupby(['joueur', 'champion']).agg(liste_champ).head(nb_champ_max)
        
        df_mois = df_filter.copy()
        
        df_mois['MOIS'] = df_filter['datetime'].dt.month
        
        df_grp_mois = df_mois.groupby(['joueur', 'MOIS']).agg(liste_champ)
        
        df_rank = df_filter.groupby(['joueur', 'tier']).agg(liste_champ)
        
        
        df_grp.columns = pd.Index([e[0] + "_" + e[1].upper() for e in df_grp.columns.tolist()])
        df_grp_champ.columns = pd.Index([e[0] + "_" + e[1].upper() for e in df_grp_champ.columns.tolist()])
        df_grp_mois.columns = pd.Index([e[0] + "_" + e[1].upper() for e in df_grp_mois.columns.tolist()])
        df_rank.columns = pd.Index([e[0] + "_" + e[1].upper() for e in df_rank.columns.tolist()])
        
        def feature_winrate(df):
            df['winrate'] = ((df['victoire_SUM'] / df['victoire_COUNT']) * 100).astype(int)
            
            df.drop(columns=['victoire_SUM'], inplace=True)
            
            df.rename(columns={'victoire_COUNT' : 'nb_games'}, inplace=True)
            
            df.sort_values('nb_games', inplace=True, ascending=False)
            
            return df
        
        df_grp = feature_winrate(df_grp)
        df_grp_champ = feature_winrate(df_grp_champ)
        df_grp_mois = feature_winrate(df_grp_mois)
        df_rank = feature_winrate(df_rank)
    
    return df_grp, df_grp_champ, df_grp_mois, discord_id, df_rank

df_grp, df_grp_champ, df_mois, discord_id, df_rank = creation_agregat(df_ranked)

tomlora
298418038460514314


In [25]:
msg_grp = ''
for index, stat in df_grp.iterrows():
   
    msg_grp += f"**{index[1]}**: Games : {stat['champion_COUNT']} | Kills : {stat['kills_SUM']} (Moy : {stat['kills_MEAN']}) | Morts : {stat['deaths_SUM']} (Moy : {stat['deaths_MEAN']}) \n"

In [26]:
df_grp_champ

kills_SUM  kills_MEAN assists_SUM  assists_MEAN deaths_SUM  \
joueur  champion                                                              
tomlora Aphelios      1086    6.205714        1083      6.188571       1058   
        Ezreal        1003    6.229814        1194      7.416149        709   
        Ashe           745    6.834862        1001      9.183486        549   
        Caitlyn        227    5.675000         255      6.375000        217   
        Draven           8    8.000000           1      1.000000         11   

                  deaths_MEAN double_SUM triple_SUM quadra_SUM penta_SUM  ...  \
joueur  champion                                                          ...   
tomlora Aphelios     6.045714        135         26          5         1  ...   
        Ezreal       4.403727         93         13          2         0  ...   
        Ashe         5.036697        110         11          3         0  ...   
        Caitlyn      5.425000         20          1          0         0  ...   
        Draven      11.000000          0          0          0         0  ...   

                 temps_avant_premiere_mort_MEAN  couronne_SUM  couronne_MEAN  \
joueur  champion                                                               
tomlora Aphelios                       7.558971         147.0       0.924528   
        Ezreal                         9.609068         190.0       1.310345   
        Ashe                           7.562936         138.0       1.408163   
        Caitlyn                       10.302750          30.0       0.750000   
        Draven                         7.050000           0.0       0.000000   

                 team_SUM  team_kills_SUM  team_kills_MEAN  team_deaths_SUM  \
joueur  champion                                                              
tomlora Aphelios       84            4835        27.628571             5172   
        Ezreal         72            4654        28.906832             4464   
        Ashe           57            3372        30.935780             3020   
        Caitlyn        21            1081        27.025000             1198   
        Draven          1              18        18.000000               38   

                 team_deaths_MEAN  champion_COUNT winrate  
joueur  champion                                           
tomlora Aphelios        29.554286             175      44  
        Ezreal          27.726708             161      53  
        Ashe            27.706422             109      59  
        Caitlyn         29.950000              40      45  
        Draven          38.000000               1       0  

[5 rows x 38 columns]

In [27]:
df_mois

kills_SUM  kills_MEAN assists_SUM  assists_MEAN deaths_SUM  \
joueur  MOIS                                                              
tomlora 8         1143    6.494318        1270      7.215909        978   
        6         1140    6.993865        1306      8.012270        800   
        9         1074    7.065789        1121      7.375000        822   
        5          960    6.530612        1005      6.836735        824   
        1          790    6.124031         916      7.100775        603   
        10         674    7.573034         707      7.943820        525   
        7          442    6.225352         490      6.901408        334   
        2          405    5.955882         512      7.529412        363   
        4          117    6.882353          99      5.823529         96   
        3           76    6.333333          72      6.000000         65   

              deaths_MEAN double_SUM triple_SUM quadra_SUM penta_SUM  ...  \
joueur  MOIS                                                          ...   
tomlora 8        5.556818        134         19          1         0  ...   
        6        4.907975        153         22          7         2  ...   
        9        5.407895        130         19          3         0  ...   
        5        5.605442        114         14          2         1  ...   
        1        4.674419         93         10          0         0  ...   
        10       5.898876         95         17          2         0  ...   
        7        4.704225         35          7          2         0  ...   
        2        5.338235         48          7          1         0  ...   
        4        5.647059         13          1          1         0  ...   
        3        5.416667          6          0          0         0  ...   

             temps_avant_premiere_mort_MEAN  couronne_SUM  couronne_MEAN  \
joueur  MOIS                                                               
tomlora 8                          8.499489         193.0       1.221519   
        6                          8.332515         195.0       1.354167   
        9                         10.570263         160.0       1.185185   
        5                          7.637143         132.0       0.992481   
        1                         11.114884         153.0       1.186047   
        10                         9.242584         101.0       1.311688   
        7                          4.167606          76.0       1.187500   
        2                         10.193382          85.0       1.250000   
        4                          8.054118          14.0       0.933333   
        3                          9.114167          11.0       0.916667   

             team_SUM  team_kills_SUM  team_kills_MEAN  team_deaths_SUM  \
joueur  MOIS                                                              
tomlora 8          89            5120        29.090909             5237   
        6          85            4912        30.134969             4423   
        9          67            4418        29.065789             4437   
        5          66            4115        27.993197             4286   
        1          61            3620        28.062016             3650   
        10         53            2722        30.584270             2592   
        7          33            1911        26.915493             1853   
        2          32            1908        28.058824             2115   
        4           7             487        28.647059              471   
        3           6             325        27.083333              348   

             team_deaths_MEAN  champion_COUNT winrate  
joueur  MOIS                                           
tomlora 8           29.755682             176      47  
        6           27.134969             163      57  
        9           29.190789             152      48  
        5           29.156463             147      41  
        1           28.294574        

In [28]:
# NOTE : Attention à la date de début du tier / rank !
df_rank

kills_SUM  kills_MEAN assists_SUM  assists_MEAN deaths_SUM  \
joueur  tier                                                                  
tomlora EMERALD       3306    6.844720        3550      7.349896       2636   
        PLATINUM      2362    6.672316        2600      7.344633       1783   
        DIAMOND       1036    6.094118        1206      7.094118        928   
                        93    7.153846         102      7.846154         54   
        GOLD            24    6.000000          40     10.000000          9   

                  deaths_MEAN double_SUM triple_SUM quadra_SUM penta_SUM  ...  \
joueur  tier                                                              ...   
tomlora EMERALD      5.457557        390         60          7         0  ...   
        PLATINUM     5.036723        285         42          9         3  ...   
        DIAMOND      5.458824        127         11          2         0  ...   
                     4.153846         16          2          1         0  ...   
        GOLD         2.250000          3          1          0         0  ...   

                 temps_avant_premiere_mort_MEAN  couronne_SUM  couronne_MEAN  \
joueur  tier                                                                   
tomlora EMERALD                        8.728861         523.0       1.219114   
        PLATINUM                       9.459322         401.0       1.211480   
        DIAMOND                        8.001471         170.0       1.069182   
                                       7.563077          19.0       1.461538   
        GOLD                          12.997500           7.0       2.333333   

                 team_SUM  team_kills_SUM  team_kills_MEAN  team_deaths_SUM  \
joueur  tier                                                                  
tomlora EMERALD       240           14039        29.066253            13994   
        PLATINUM      166           10268        29.005650            10044   
        DIAMOND        87            4754        27.964706             4967   
                        4             375        28.846154              333   
        GOLD            2             102        25.500000               74   

                 team_deaths_MEAN  champion_COUNT winrate  
joueur  tier                                               
tomlora EMERALD         28.973085             483      49  
        PLATINUM        28.372881             354      51  
        DIAMOND         29.217647             170      44  
                        25.615385              13      61  
        GOLD            18.500000               4      75  

[5 rows x 38 columns]